In [2]:
pwd

u'/Users/linzeyang'

In [3]:
cd Desktop/BIA-660/Final Project/data

/Users/linzeyang/Desktop/BIA-660/Final Project/data


In [4]:
import pandas as pd
import re
import datetime


# =========== Pre-process ==========
df = pd.read_csv('201612-citibike-tripdata.csv')

# Missing values in columns
print(df.isnull().sum())


# Snake_case the columns
def camel_to_snake(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).replace(' ','').lower()
df.columns = [camel_to_snake(col) for col in df.columns]

# Parse start_time and stop_time
try:
    df['start_datetime'] = [datetime.datetime.strptime(x, '%m/%d/%Y %H:%M:%S') for x in df.start_time]
except ValueError:
    df['start_datetime'] = [datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') for x in df.start_time]
df['start_day'] = [x.strftime('%Y-%m-%d') for x in df.start_datetime]

# =========== Group by date and station ===========
df['bikein'] = 1
df['bikeout'] = 1

df_bikeout = df.groupby(by=['start_stationid', 'start_day'])['bikeout'].count()
df_bikeout = pd.DataFrame(df_bikeout)
df_out = df_bikeout.reset_index(drop = False)
df_out.rename_axis({'start_stationid':'station_id'}, axis='columns', inplace=True)
print(df_out.head())

df_bikein = df.groupby(by=['end_stationid', 'start_day'])['bikein'].count()
df_bikein = pd.DataFrame(df_bikein)
df_in = df_bikein.reset_index(drop = False)
df_in.rename_axis({'end_stationid':'station_id'}, axis='columns', inplace=True)
print(df_in.head())


# =========== Bike loss per day for each station ===========
# Merge two DFs by station_id
df_dayloss = df_out.merge(right=df_in, left_on=['station_id', 'start_day'], right_on=['station_id', 'start_day'],
                          how='outer')
df_dayloss = df_dayloss.sort_values(['station_id', 'start_day'])
df_dayloss = df_dayloss.fillna(0)
df_dayloss['bike_loss'] = df_dayloss['bikeout'] - df_dayloss['bikein']
print(df_dayloss)

Trip Duration                  0
Start Time                     0
Stop Time                      0
Start Station ID               0
Start Station Name             0
Start Station Latitude         0
Start Station Longitude        0
End Station ID                 0
End Station Name               0
End Station Latitude           0
End Station Longitude          0
Bike ID                        0
User Type                   5388
Birth Year                 39704
Gender                         0
dtype: int64
   station_id   start_day  bikeout
0          72  2016-12-01      100
1          72  2016-12-02       92
2          72  2016-12-03       53
3          72  2016-12-04       47
4          72  2016-12-05       78
   station_id   start_day  bikein
0          72  2016-12-01      90
1          72  2016-12-02      85
2          72  2016-12-03      50
3          72  2016-12-04      60
4          72  2016-12-05      62
       station_id   start_day  bikeout  bikein  bike_loss
0            72.0  2

In [5]:
# create weekday column
#wekday_name=pd.to_datetime(df_dayloss.start_day).dt.weekday_name   #name
df_wekday_index=pd.to_datetime(df_dayloss.start_day).dt.weekday   # index Monday=0, Sunday=6

In [6]:
# Append weekday column to df_dayloss
df_dayloss['wek_index']=df_wekday_index

In [7]:
df_dayloss

,station_id,start_day,bikeout,bikein,bike_loss,wek_index
0,72.0,2016-12-01,100.0,90.0,10.0,3
1,72.0,2016-12-02,92.0,85.0,7.0,4
2,72.0,2016-12-03,53.0,50.0,3.0,5
3,72.0,2016-12-04,47.0,60.0,-13.0,6
4,72.0,2016-12-05,78.0,62.0,16.0,0
5,72.0,2016-12-06,72.0,72.0,0.0,1
6,72.0,2016-12-07,80.0,84.0,-4.0,2
7,72.0,2016-12-08,74.0,84.0,-10.0,3
8,72.0,2016-12-09,68.0,58.0,10.0,4
9,72.0,2016-12-10,38.0,34.0,4.0,5


In [8]:
# =========== count docks ============
from six.moves import urllib

import json
import collections
import itertools
jsonurl = urllib.request.urlopen('https://gbfs.citibikenyc.com/gbfs/en/station_status.json')
text = json.loads(jsonurl.read()) # <-- read from it
data=text['data']
#print(len(data['stations']))   664 stations
#print(data['stations'][0]['station_id']) # station id

# station id Extract:
total_stations_id = []
for i in range(len(data['stations'])):
    id=data['stations'][i]['station_id']
    total_stations_id.append(id)
total_stations_id=[int(x) for x in total_stations_id]
#print('station_id list:'+str(total_stations_id))

# num_bikes_available Extract:
total_num_bikes_available = []
for i in range(len(data['stations'])):
    num_bikes_available = data['stations'][i]['num_bikes_available']
    total_num_bikes_available.append(num_bikes_available)
#print('bikes availble:'+str(total_num_bikes_available))

# num_docks_available Extract:
total_docks_available = []
for i in range(len(data['stations'])):
    docks_availble=data['stations'][i]['num_docks_available']
    total_docks_available.append(docks_availble)
#print('docks availble:'+str(total_docks_available))

total_docks=[x + y for x, y in zip(total_num_bikes_available, total_docks_available)]
#print('total docks for each station:'+str(total_docks))
#to_dict

new_dict = dict(zip(total_stations_id,total_docks))
print('zip station_id and total docks:'+str(new_dict))


zip station_id and total docks:{72: 36, 79: 32, 82: 0, 83: 59, 116: 39, 119: 19, 120: 19, 127: 30, 128: 29, 137: 0, 143: 23, 144: 19, 146: 37, 147: 0, 150: 31, 151: 33, 152: 29, 153: 54, 157: 23, 161: 35, 164: 46, 167: 44, 168: 46, 173: 50, 174: 30, 195: 44, 212: 26, 216: 23, 217: 37, 223: 32, 224: 0, 225: 37, 228: 55, 229: 23, 232: 22, 236: 37, 237: 39, 238: 30, 239: 29, 241: 23, 242: 23, 243: 31, 244: 30, 245: 22, 247: 18, 248: 23, 249: 26, 251: 26, 252: 33, 253: 53, 254: 29, 257: 39, 258: 22, 259: 39, 260: 31, 261: 27, 262: 24, 264: 27, 265: 33, 266: 24, 267: 55, 268: 25, 270: 23, 274: 30, 275: 19, 276: 25, 278: 19, 279: 36, 280: 31, 281: 56, 282: 27, 284: 40, 285: 0, 289: 19, 291: 19, 293: 55, 295: 23, 296: 35, 297: 27, 301: 37, 302: 23, 303: 30, 304: 34, 305: 31, 306: 35, 307: 29, 308: 27, 309: 39, 310: 36, 311: 31, 312: 31, 313: 23, 314: 39, 315: 29, 316: 43, 317: 27, 319: 31, 320: 38, 321: 27, 322: 3, 323: 36, 324: 51, 325: 34, 326: 27, 327: 39, 328: 22, 330: 39, 331: 25, 332: 2

In [106]:
# Number of stations:
print 'Unique stations: '+str(len(new_dict))

Unique stations: 664


In [9]:
# append docks to df_dayloss
df_dayloss['docks'] = df_dayloss['station_id'].map(new_dict)

In [98]:
df_dayloss[1:5]

,station_id,start_day,bikeout,bikein,bike_loss,wek_index,docks,holiday_idx
1,72.0,2016-12-02,92.0,85.0,7.0,4,36.0,0
2,72.0,2016-12-03,53.0,50.0,3.0,5,36.0,0
3,72.0,2016-12-04,47.0,60.0,-13.0,6,36.0,0
4,72.0,2016-12-05,78.0,62.0,16.0,0,36.0,0


In [11]:
# Append holiday column to df_dayloss
holiday = ['2016-01-01', '2016-01-18', '2016-02-12', '2016-02-15',
          '2016-05-08', '2016-05-30', '2016-06-19', '2016-07-04',
          '2016-09-05', '2016-10-10', '2016-11-11', '2016-11-24',
          '2016-12-26']
df_dayloss['holiday_idx'] = df_dayloss['start_day'].apply(lambda x: 1 if (x in holiday) else 0)

In [22]:
# Find out which column has missing data
# Result: only 'docks' column
pd.isnull(df_dayloss).sum() > 0

station_id     False
start_day      False
bikeout        False
bikein         False
bike_loss      False
wek_index      False
docks           True
holiday_idx    False
dtype: bool

In [47]:
# Extract rows that has missing data and deal with it 
# Result: missing rows only accounts for 2% of raw data
missing_rows=df_dayloss[pd.isnull(df_dayloss).any(axis=1)]
ratio_missing_rows=float(len(missing_rows))/len(df_dayloss)
ratio_missing_rows

0.02247191011235955

In [84]:
# Missing station ids listed: maybe didn't detect by json
# Will delete them and do regression first.
missing_station_id=missing_rows.station_id
unique_missing_station_id=set(missing_station_id)

In [85]:
for i in unique_missing_station_id:
    print i
    #df[df['start_stationid']==i][0]

3264.0
2017.0
3099.0
3237.0
3079.0
3240.0
3017.0
298.0
3239.0
3084.0
3245.0
3246.0
367.0
3250.0
3219.0
3265.0
3098.0
475.0
3036.0
3230.0
255.0


In [100]:
df[df['start_stationid']==2017].iloc[0]

trip_duration                             1314
start_time                 2016-12-01 05:10:33
stop_time                  2016-12-01 05:32:28
start_stationid                           2017
start_station_name             E 43 St & 2 Ave
start_station_latitude                 40.7502
start_station_longitude               -73.9712
end_stationid                              459
end_station_name              W 20 St & 11 Ave
end_station_latitude                   40.7467
end_station_longitude                 -74.0078
bikeid                                   24311
user_type                           Subscriber
birth_year                                1951
gender                                       1
start_datetime             2016-12-01 05:10:33
start_day                           2016-12-01
bikein                                       1
bikeout                                      1
Name: 363, dtype: object

In [58]:
# Delete missing rows
df_dayloss=df_dayloss.dropna(axis=0)

In [75]:
# OLS regression 
from sklearn.linear_model import LinearRegression
X,Y=df_dayloss.ix[:,5:8],df_dayloss.ix[:,4]   # Assign features to be X and target to be Y.
mdl = LinearRegression().fit(X,Y)
m = mdl.coef_
b = mdl.intercept_
print 'Y={}X+{}'.format(m,b)

Y=[ 0.01536283  0.00318018  0.09493163]X+-0.18582647622


In [124]:
# scrape weather -- haven't complete
from selenium import webdriver
driver = webdriver.Chrome('/Users/linzeyang/Desktop/BIA-660/chromedriver')
driver.get('https://www.wunderground.com/history/airport/KNYC/2016/6/1/MonthlyHistory.html?req_city=New+York&req_state=NY&req_statename=&reqdb.zip=10001&reqdb.magic=11&reqdb.wmo=99999')
to_input = driver.find_elements_by_xpath("//div[@id='content-wrap']/div[@id='inner-wrap']")
driver.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=57.0.2987.133)
  (Driver info: chromedriver=2.27.440174 (e97a722caafc2d3a8b807ee115bfb307f7d2cfd9),platform=Mac OS X 10.11.6 x86_64)


In [122]:
len(to_input)

1